# Stable Diffusion finetuning
Ноутбук с примером обучения StableDiffusionInpaint для использования полученных весов в генераторе синтетических аугментаций

In [ ]:
import os
import json
import math

import numpy as np
from pathlib import Path
from PIL import Image
from IPython.display import clear_output
from tqdm.auto import tqdm
from datasets import Dataset, load_from_disk

from syntgenerator import AugmentationGenerator, SDItrainer

## Подготовка данных для обучения
Для обучения StableDiffusionInpaint необходимо подготовить датасет, каждый экземпляр которого будет хранить оригинальное изображение, маску и промпт(текстовую подсказку для обучения). Такой датасет будет подготовлен из датасета в формате COCO для детекции объектов.

В данном разделе будут вырезаться изображения размером 512x512 пикселей вокруг размеченных bbox'ов и создаваться маска с белым прямоугольником на месте bbox'а. Для каждого изображения необходимо написать несколько текстовый подсказок, по которым будет учиться новая модель.
Формат полученного датасета выглядит так:

    {
        'images': list<PIL image>
        'masks': list<PIL image>
        'text': list<str>
    }

In [ ]:
COCO_DIR = './data/example.json' #путь до JSON файла с COCO разметкой
IMGS_DIR = './data/example/' #путь до директории с изображениями

In [ ]:
images = []
masks = []

with open(COCO_DIR, 'r') as f:
    coco = json.load(f)
    for ann in coco['annotations']:
            img_name = coco['images'][ann['image_id']]['file_name'].split('/')[-1]
            if os.path.exists(IMGS_DIR+img_name):
                img = Image.open(IMGS_DIR+img_name)
                w, h = img.size
                bbox = ann['bbox']
                bbox[2] += bbox[0]
                bbox[3] += bbox[1]
                att_area, mask, _, _ = AugmentationGenerator.generate_attention_area(img=img, bbox=bbox, aa_size=512)
                images += [att_area]
                masks += [mask]
                
len(images), len(masks)

In [ ]:
dataset_dict = {
    'images': [],
    'masks': [],
    'text': []
}

In [ ]:
nn_prompts = 1 #количество промптов для каждого изображения
for i, img in enumerate(images):
    display(img)
    for _ in range(nn_prompts):
        prompt = input()
        dataset_dict['text'] += [prompt]
        dataset_dict['images'] += [img]
        dataset_dict['masks'] += [masks[i]]
    clear_output()

In [ ]:
DATASET_DIR = './dataset_example' #директория для сохранения датасета
inpaint_dataset = Dataset.from_dict(dataset_dict)
inpaint_dataset.save_to_disk(DATASET_DIR)

## Обучение

In [ ]:
pretrained_model_name_or_path = 'stabilityai/stable-diffusion-2-inpainting'
output_dir = './sd_inpaint_finetune'

trainer = SDItrainer(pretrained_model_name_or_path, output_dir)

In [ ]:
data_dir = 'dataset_example'
train_batch_size = 1
max_train_steps = 400
resolution = 512

trainer(data_dir, train_batch_size, max_train_steps, resolution)